# Code to Save Mesh Data and Geodesic Solver
Saves out the data for the female and male meshes and the solvers for preloading of the data. This will greatly speed up calculations

In [1]:
import numpy as np
import pandas as pd
import potpourri3d as pp3d

## Function to save out data from mesh obj file

In [2]:
def get_mesh_data(mesh_name: str) -> None:
    '''
    Saves out the mesh data from an obj file

    Parameters
    ----------
    mesh_name: str
        The name of the mesh in the Model's folder
    '''
    # read in the mesh
    mesh_verticies, mesh_faces =\
        pp3d.read_mesh("../Models/" + mesh_name + " UV Mapped.obj")

    # Load in the text version of the model
    mesh_data =\
        pd.read_csv("../Models/" + mesh_name + " UV Mapped as Text.txt",
                    names=["Type", "Point 1", "Point 2", "Point 3"],
                    delim_whitespace=True, dtype=str)
    grouped_mesh_data = mesh_data.groupby(["Type"])

    # Get the UV data
    uv_data = grouped_mesh_data.get_group("vt")
    uv_data = uv_data.astype(
        {"Point 1": float, "Point 2": float, "Point 3": float})
    uv_data.drop("Type", axis=1, inplace=True)
    uv_data.drop("Point 3", axis=1, inplace=True)
    uv_data.rename(columns={"Point 1": "x", "Point 2": "y"},
                        inplace=True)
    uv_data.reset_index(drop=True, inplace=True)
    uv_data.index += 1
    uv_array = uv_data.values  # convert the uv data to a numpy array

    # Get the face data
    face_data = grouped_mesh_data.get_group("f")
    split_face_data_1 = pd.DataFrame()
    split_face_data_1[["vertex", "uv", "normal"]] =\
        face_data["Point 1"].str.split("/", expand=True)
    split_face_data_2 = pd.DataFrame()
    split_face_data_2[["vertex", "uv", "normal"]] =\
        face_data["Point 2"].str.split("/", expand=True)
    split_face_data_3 = pd.DataFrame()
    split_face_data_3[["vertex", "uv", "normal"]] =\
        face_data["Point 3"].str.split("/", expand=True)
    split_face_data = pd.concat(
        [split_face_data_1, split_face_data_2, split_face_data_3])
    split_face_data = split_face_data.astype(
        {"vertex": float, "uv": float, "normal": float})
    face_data = split_face_data.reset_index(drop=True)
    face_data.index += 1

    np.savez_compressed("../Data/" + mesh_name.lower() + " mesh data.npz",
             mesh_verticies=mesh_verticies, mesh_faces=mesh_faces,
             uv_array=uv_array, face_data=face_data)

## Example for the Male Right Arm

In [3]:
mesh_name = "Male Right Arm"
get_mesh_data(mesh_name)

In [4]:
imported_data = np.load("../Data/male right arm mesh data.npz")
print(imported_data["uv_array"])

[[0.102348 0.273586]
 [0.931549 0.540414]
 [0.106193 0.042448]
 ...
 [0.151023 0.816322]
 [0.151804 0.817039]
 [0.152532 0.817768]]


## Face data needs to be imported and converted to a pandas Dataframe

In [5]:
imported_face_data = pd.DataFrame(imported_data["face_data"], columns=["vertex", "uv", "normal"])
imported_face_data.head()

,vertex,uv,normal
0,25606.0,26613.0,1.0
1,25607.0,26614.0,2.0
2,25608.0,26615.0,3.0
3,25609.0,26616.0,4.0
4,25610.0,26617.0,5.0


## Save out the rest of the data

In [ ]:
get_mesh_data("Male Left Arm")
get_mesh_data("Female Right Arm")
get_mesh_data("Female Left Arm")